# RAG 시스템 - 문서 저장 (Document Storage)

이 노트북은 RAG(Retrieval-Augmented Generation) 시스템에서 문서를 벡터 데이터베이스에 저장하는 과정을 설명합니다.

## 주요 구성요소
1. 텍스트 분할 (Text Splitting)
2. 임베딩 생성 (Embedding Generation)
3. 벡터 저장소 (Vector Storage)

## 사전 요구사항
- ChromaDB가 실행 중이어야 합니다
- 필요한 Python 패키지가 설치되어 있어야 합니다

## 1. 필요한 라이브러리 임포트

RAG 시스템 구현에 필요한 핵심 라이브러리들을 임포트합니다:
- `langchain_community.vectorstores`: 벡터 저장소 관리
- `langchain.text_splitter`: 문서를 작은 청크로 분할
- `langchain_community.embeddings`: 텍스트를 벡터로 변환

In [ ]:
import json
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

## 2. ChromaDB 설정

ChromaDB 연결 및 데이터 저장을 위한 기본 설정을 정의합니다:
- `CHROMA_HOST`: ChromaDB 서버 주소
- `CHROMA_PORT`: 서버 포트
- `PERSIST_DIR`: 데이터 영구 저장 경로

In [ ]:
CHROMA_HOST = "localhost"
CHROMA_PORT = "8090"
PERSIST_DIR = "./data"  # ChromaDB 데이터 저장 경로

## 3. 임베딩 모델 설정

텍스트를 벡터로 변환하는 임베딩 모델을 설정합니다.
- 모델: nomic-ai/nomic-embed-text-v1
- CPU 기반 처리
- 정규화된 임베딩 생성

In [ ]:
def get_embeddings():
    """한국어에 최적화된 임베딩 모델 생성"""
    return HuggingFaceEmbeddings(
        model_name="nomic-ai/nomic-embed-text-v1",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

## 4. 문서 저장 함수

문서를 ChromaDB에 저장하는 핵심 함수입니다. 다음 단계로 진행됩니다:

1. 문서 텍스트 추출
2. 한글 최적화 텍스트 분할
3. 임베딩 생성
4. ChromaDB 저장

특징:
- 한글 문서에 최적화된 분할 규칙 사용
- 300자 단위로 청크 생성
- 50자 오버랩으로 문맥 유지

In [ ]:
def store_documents_in_chroma(documents, collection_name="rag_test"):
    """문서를 벡터로 변환하여 ChromaDB에 저장"""
    try:
        # 문서 텍스트 추출
        texts = []
        for doc in documents:
            if hasattr(doc, 'page_content'):
                texts.append(doc.page_content)
            elif isinstance(doc, str):
                texts.append(doc)
            else:
                raise ValueError(f"지원하지 않는 문서 형식: {type(doc)}")

        # 한글 문서에 최적화된 텍스트 스플리터 설정
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n", "\n", ".", "!", "?", "。", "！", "？", " ", ""],
            chunk_size=300,
            chunk_overlap=50,
            length_function=len,
            keep_separator=False,
            is_separator_regex=False
        )
        
        # 문서를 청크로 분할
        splits = []
        for text in texts:
            if isinstance(text, str) and text.strip():
                doc_splits = text_splitter.split_text(text)
                splits.extend(doc_splits)

        if not splits:
            raise ValueError("유효한 텍스트가 없습니다")

        # 임베딩 모델 사용
        embedding_model = get_embeddings()

        # Chroma 인스턴스 생성 및 저장
        vector_db = Chroma.from_texts(
            texts=splits,
            embedding=embedding_model,
            collection_name=collection_name,
            persist_directory=PERSIST_DIR
        )

        print(f"✅ {len(splits)}개의 청크가 성공적으로 저장되었습니다.")
        return vector_db

    except Exception as e:
        print(f"❌ 벡터DB 저장 실패: {str(e)}")
        return None

## 5. 예제: 문서 저장

샘플 문서를 사용하여 저장 기능을 테스트합니다.
- 한국의 계절에 대한 간단한 설명문
- 각 문장은 자동으로 적절한 크기로 분할됨
- 저장 결과 확인 가능

In [ ]:
# 예제: 문서 저장하기
sample_documents = [
    "한국의 계절은 봄, 여름, 가을, 겨울로 나뉩니다. 각 계절은 고유한 특징을 가지고 있습니다.",
    "봄에는 꽃이 피고 날씨가 따뜻해집니다. 벚꽃과 진달래가 대표적인 봄꽃입니다.",
    "여름은 덥고 습합니다. 장마철이 있으며, 해수욕장이나 계곡을 찾는 사람들이 많습니다."
]

vector_db = store_documents_in_chroma(sample_documents)